In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0, '../')

In [3]:
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import pauli_error, depolarizing_error

def get_noise(p_err):

    error_gate1 = pauli_error([('X', p_err/2), ('Z', p_err/2), ('I', 1 - p_err)])
    error_gate2 = error_gate1.tensor(error_gate1)

    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error_gate1, "measure") # measurement error is applied to measurements
    noise_model.add_all_qubit_quantum_error(error_gate1, ["u1", "u2", "u3"]) # single qubit gate error is applied to x gates
    noise_model.add_all_qubit_quantum_error(error_gate2, ["cx"]) # two qubit gate error is applied to cx gates
        
    return noise_model

In [4]:
from qiskit import execute, Aer
from Encoder import SurfaceCode
from fitters import GraphDecoder

In [26]:
def full_loop(d, T):
    code = SurfaceCode(d, T)
    decoder = GraphDecoder(d, T)

    # noise_model = get_noise(0.01)
    counts = execute(code.circuit['0'], Aer.get_backend('qasm_simulator'), shots=1).result().get_counts()
    X_errors, Z_errors = code.extract_nodes(code.process_results(counts))

    print("Raw readout:", counts)
    print("Parsed X:", X_errors)
    print("Parsed Z:", Z_errors)

    X_errors = [node for node in X_errors if node[0] != -2]
    Z_errors = [node for node in Z_errors if node[0] != -2]

    if X_errors:
        error_graph, paths = decoder.make_error_graph(X_errors, "X")
        matching_graph = decoder.matching_graph(error_graph,'X')
        matches = decoder.matching(matching_graph,'X')
        flips_x = decoder.calculate_qubit_flips(matches, paths,'X')
    else:
        flips_x = {}

    if Z_errors:
        error_graph, paths = decoder.make_error_graph(Z_errors, "Z")
        matching_graph = decoder.matching_graph(error_graph,'Z')
        matches = decoder.matching(matching_graph,'Z')
        flips_z = decoder.calculate_qubit_flips(matches, paths,'Z')
    else:
        flips_z = {}

    print("----------------")
    print("Decoder results:")
    print(decoder.net_qubit_flips(flips_x, flips_z))

In [31]:
for i in range(10):
    print("====Run", i, "====")
    full_loop(3, 1)
    print("=============", "\n")

====Run 0 ====
Raw readout: {'000110110 00000000': 1}
Parsed X: [(-2, 1, 0), (-2, 2, 0)]
Parsed Z: [(-2, 2, 0), (-2, 2, 1)]
Decoder results:
{}

====Run 1 ====
Raw readout: {'011101101 00000000': 1}
Parsed X: [(-2, 0, 2), (-2, 1, 0), (-2, 1, 2), (-2, 2, 0), (-2, 2, 2)]
Parsed Z: [(-2, 0, 1), (-2, 0, 2), (-2, 2, 0), (-2, 2, 2)]
Decoder results:
{}

====Run 2 ====
Raw readout: {'110000000 00000101': 1}
Parsed X: [(-2, 0, 0), (0, 1.5, 0.5), (0, 2.5, 1.5)]
Parsed Z: [(-2, 0, 0), (-2, 0, 1)]
Decoder results:
Physical Qubit: (2.0, 1.0)
Error: X at time: 0
{(2.0, 1.0): array([[0, 1],
       [1, 0]])}

====Run 3 ====
Raw readout: {'101101110 00000100': 1}
Parsed X: [(-2, 0, 0), (-2, 0, 2), (-2, 1, 0), (-2, 1, 2), (-2, 2, 0), (0, 1.5, 0.5)]
Parsed Z: [(-2, 0, 0), (-2, 0, 2), (-2, 2, 0), (-2, 2, 1)]
Decoder results:
Physical Qubit: (1.0, 0.0)
Error: X at time: 0
{(1.0, 0.0): array([[0, 1],
       [1, 0]])}

====Run 4 ====
Raw readout: {'000000000 10100100': 1}
Parsed X: [(0, -0.5, 0.5), (0, 0.5,